In [6]:
import torch
from torch import nn, optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

tf = transforms.Compose([
    transforms.Resize((128,128)),
    transforms.ToTensor(),
    transforms.Normalize([0.5,0.5,0.5],[0.5,0.5,0.5])
])

train_dl = DataLoader(
    datasets.ImageFolder('./Training',tf),
    batch_size=32, shuffle=True,num_workers=4,pin_memory=True
)

test_dl = DataLoader(
    datasets.ImageFolder('./Testing',tf),
    batch_size=32, shuffle=False,num_workers=4,pin_memory=True
)

In [7]:
model = nn.Sequential(
    nn.Conv2d(3,32,3,1,1),nn.ReLU(),nn.MaxPool2d(2),
    nn.Conv2d(32,64,3,1,1),nn.ReLU(),nn.MaxPool2d(2),
    nn.Conv2d(64,128,3,1,1),nn.ReLU(),nn.MaxPool2d(2),
    nn.Flatten(),
    nn.Linear(128*16*16,256),nn.ReLU(),nn.Dropout(0.5),
    nn.Linear(256,4) #4 classes
).to(device)


In [8]:
opt = optim.Adam(model.parameters(),1e-4)
loss_fn = nn.CrossEntropyLoss()

In [ ]:
model.train()
for epoch in range(25):
    running_loss = 0

    for x,y in train_dl:
        opt.zero_grad()

        loss = loss_fn(model(x.to(device)), y.to(device))
        running_loss += loss
        opt.step()

    print(f'Epoch {epoch+1} : Loss was {running_loss}')


C:\Users\insec\PycharmProjects\BrainTumor\.venv\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


In [ ]:
model.eval()
test_loss,correct = 0.0

with torch.no_grad():
    for x,y in test_dl:
        x,y = x.to(device),y.to(device)

        logits = model(x)
        test_loss += loss_fn(logits,y).item()*y.size(0)

        preds = logits.argmax(dim=1)
        correct += (preds == y).sum().item()

test_loss /= len(test_dl.dataset)
accuracy = 100.0 * correct / len(test_dl.dataset)